In [ ]:
#For data processing
import pandas as pd
import numpy as np
import datetime as dt

#For data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#For models
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# IMPORTING IMPORTANT LIBRARIES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import preprocessing 

#version verify
from platform import python_version

In [ ]:
# Check versions of modules used 
print("numpy: {}".format(np.__version__))
print("pandas: {}".format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))
print("seaborn: {}".format(sns.__version__))
#print("yahoo_finance: {}".format(yf.__version__))
print("python version:",python_version())

In [ ]:

# # fix_yahoo_finance is used to fetch data 
# import yfinance as yf
# yf.pdr_override()


# # input
# symbol = 'RELIANCE.NS'
# start = '2017-01-01'
# end = '2018-01-01'


# # dataframe
# dataset = yf.download(symbol,start,end)



# # View the last 5 rows
# print('Last 5 Rows')
# print(df.tail())

#Read input file

raw_data = pd.read_csv("C:\\Users\\tcgra\\Documents\\LJMU_Code\Datasets\\infosys_stock.csv")
raw_data.head(5)


In [ ]:
raw_data.info()

In [ ]:
# Inspect the index 
raw_data.index

In [ ]:
raw_data.describe() # Statistics

In [ ]:
print("Stock dataset has {} data points with {} variables each.".format(*raw_data.shape))

In [ ]:
# if it's a larger dataset and the visualization takes too long can do this.
# % of missing.
for col in raw_data.columns:
    pct_missing = np.mean(raw_data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
#verify if there is any null values in the dataset
print(raw_data.isnull().sum())

#verify if there is any duplicate records
print(raw_data.duplicated())

In [ ]:
dt_data = raw_data
dt_data = raw_data.set_index(['Date'])

In [ ]:
import matplotlib.pyplot as plt   # Import matplotlib
# This line is necessary for the plot to appear in a Jupyter notebook
%matplotlib inline
# Control the default size of figures in this Jupyter notebook
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)   # Change the size of plots
 
#dt_data['Date','Open','High','Low','Close'].plot(grid = True) # Plot the adjusted closing price of AAPL
dt_data[['Open','High','Low','Close']].plot(figsize=(10,5))
plt.title('Open High Low Close')
plt.ylabel("Price")
plt.show()

In [ ]:
raw_data.boxplot(column=['Open','High','Low','Close'])

In [ ]:
Q1=raw_data['Open'].quantile(0.25)
Q3=raw_data['Open'].quantile(0.75)
IQR=Q3-Q1
print(Q1)
print(Q3)
print(IQR)
Lower_Whisker = Q1 - 1.5*IQR
Upper_Whisker = Q3+1.5*IQR
print(Lower_Whisker, Upper_Whisker)
raw_data.shape

In [ ]:
raw_data = raw_data[raw_data['Open']< Upper_Whisker]
raw_data.shape

In [ ]:
raw_data_float = pd.DataFrame(raw_data,columns=['Open','High','Low','Close'])


for i, col in enumerate(raw_data_float.columns):
    plt.figure(i)
    sns.distplot(raw_data[col])

In [ ]:
import seaborn as sns
#sns.set_style('White')
cmap = sns.diverging_palette(220, 10, as_cmap=True)

plt.figure(figsize=(15,10))
sns.heatmap(raw_data[1:].corr(), cmap='Blues')
plt.show()

# Return Distribution Analysis

In [ ]:
# Convert into another dataset
df = raw_data.copy()
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index(['Date'])

#returns = raw_data['Adj Close'].pct_change()[1:].dropna()

stock_daily_returns = df['Adj Close'].pct_change()[1:].dropna()
stock_monthly_returns = df['Adj Close'].resample('M').ffill().pct_change()[1:].dropna()
stock_yearly_returns = df['Adj Close'].resample('Y').ffill().pct_change()[1:].dropna()


In [ ]:
returns = stock_yearly_returns

import statistics as st
import numpy as np
from scipy.stats import skew,kurtosis,norm,skewtest,kurtosistest

print('Mean of returns:', np.mean(returns))
print('SD of returns:', np.std(returns))


print(skew(returns))
print(skewtest(returns))
print(kurtosis(returns))
print(kurtosistest(returns))

from statsmodels.stats.stattools import jarque_bera

_, pvalue, _, _ = jarque_bera(returns)

if pvalue > 0.05:
    print('The returns are likely normal.')
else:
    print('The returns are likely not normal.')



In [ ]:
# boxplot

plt.boxplot(stock_daily_returns,labels=["Monthly Returns"])
plt.show()


In [ ]:
data_to_plot = [stock_daily_returns, stock_monthly_returns, stock_yearly_returns]
fig = plt.figure()
# Create an axes instance
ax = fig.add_axes([0,0,1,1])
# Create the boxplot
bp = ax.boxplot(data_to_plot)
plt.show()

# Skewness & Kurtosis

In [ ]:
print("Excess kurtosis of returns: ", stats.kurtosis(returns))

In [ ]:
from statsmodels.stats.stattools import jarque_bera

_, pvalue, _, _ = jarque_bera(returns)

if pvalue > 0.05:
    print('The returns are likely normal.')
else:
    print('The returns are likely not normal.')

In [ ]:

# Plot some example distributions stock's returns
returns = stock_daily_returns
xs = np.linspace(-6,6, (len(raw_data)-1))
normal = stats.norm.pdf(xs)
plt.plot(returns,stats.laplace.pdf(returns), label='Leptokurtic')
print('Excess kurtosis of leptokurtic distribution:', (stats.laplace.stats(returns)))
plt.plot(returns, normal, label='Mesokurtic (normal)')
print('Excess kurtosis of mesokurtic distribution:', (stats.norm.stats(returns)))
plt.plot(returns,stats.cosine.pdf(returns), label='Platykurtic')
print('Excess kurtosis of platykurtic distribution:', (stats.cosine.stats(returns)))
plt.legend()

# Distribution Analysis of Returns

In [ ]:
returns = stock_monthly_returns
t = np.linspace(0.01,0.99,1000)
q1 = np.quantile(returns,t)
q2 = norm.ppf(t,loc=np.mean(returns),scale=np.std(returns))
plt.plot(q1,q2)
plt.plot([min(q1),max(q1)],[min(q2),max(q2)])
plt.xlim((min(q1),max(q1)))
plt.ylim((min(q2),max(q2)))
plt.xlabel("Daily returns")
plt.ylabel("Normal distribution")
plt.show()

In [ ]:
values = stock_monthly_returns
x = np.linspace(values.min(), values.max(), len(values))
loc, scale = stats.norm.fit(values)
param_density = stats.norm.pdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(values, bins=30, normed=True)
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')
ax.set_title("Probability Density Function")

values = stock_monthly_returns
x = np.linspace(values.min(), values.max(), len(values))
loc, scale = stats.norm.fit(values)
param_density = stats.norm.cdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)

fig, ax = plt.subplots(figsize=(10, 6))
#ax.hist(values, bins=30, normed=True)
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')
ax.set_title("Cumulative Probability Distribution")

In [ ]:
stock_daily_returns = df['Adj Close'].pct_change()
plt.stem(raw_data['Date'], stock_daily_returns)
(dt_data['Volume']/1000000).plot(figsize = (15, 7.5), 
                                 color = 'green', 
                                 alpha = 0.5)
plt.show

In [ ]:
from scipy.stats import binom

n = len(raw_data['Returns'])
p = raw_data['Returns'].mean()
k = np.arange(0,21)
binomial = binom.pmf(k,n,p)

plt.plot(k, binomial, 'o-')
plt.title("Binomial: n=%i, p=%.2f" % (n,p), fontsize=15)
plt.xlabel("Number of successes", fontsize=15)
plt.ylabel("Probability of successes", fontsize=15)
plt.show()

In [ ]:
s = np.random.uniform(values.min(), values.max(), len(values))

import matplotlib.pyplot as plt
count, bins, ignored = plt.hist(s, 15, density=True)
plt.plot(bins, np.ones_like(bins), linewidth=2, color='r')
plt.show()

In [ ]:
rate = 3 # Error Rate
n = np.arange(0,10) # Number of Trials
y = stats.poisson.pmf(n, rate)
y

In [ ]:

plt.plot(n, y, 'o-')
plt.title('Poisson: $\lambda$ =%i' % rate)
plt.ylabel('Probability Error')
plt.xlabel('Number of Errors (out of 100 trials)')
plt.show()

Compute Alpha & Beta